# Setup and Organization

DATA FROM: https://github.com/fivethirtyeight/uber-tlc-foil-response/blob/master/README.md

1. double check data quality for 2014 files
2. combine 2014 files and export for SQL analysis
3. check 2015 file quality

## Checking 2014 Data
- Check/update dtypes
- Make sure each file goes to end of month
- Check for nulls

In [ ]:
import pandas as pd
import glob

files = glob.glob('2014/uber-raw-data*.csv')
print(f"found {len(files)} files")

### Checking dtypes

In [ ]:
for file in files:
    print(file) # prints file name
    df = pd.read_csv(file) # converts to interactive df
    print(df.dtypes) # finds dtype for each column 

Lat and Lon looks good, Date/Time should be converted to datetime.
Base we'll leave for now, may change in SQL.

### Updating dtype & checking ensuring full month of data

In [ ]:
# updating dtype and checking max date (ensure its end of the month)
for file in files:
    max_date = pd.to_datetime(pd.read_csv(file)['Date/Time']).max()
    df = pd.read_csv(file)
    print(f"{file}: {max_date}, monthly rides = {len(df)}")

### Checking for nulls

In [ ]:
# first look at full files, will dive in if necessary
for file in files:
    df = pd.read_csv(file)
    num_null = df.isnull().sum().sum()
    print(f"{file}: {num_null}")

no nulls

## Merging 2014 files to 1 csv

In [ ]:
# Creates a list of DataFrames using the files from the 'Uber/tables' directory
dfs = [pd.read_csv(file, parse_dates=['Date/Time']) for file in files]

# combines list into one DataFrame
combined = pd.concat(dfs, ignore_index=True).sort_values('Date/Time')

# make sure matches with earlier row counts
print(combined.shape)

In [ ]:
# exports new combined DataFrame into a csv
combined.to_csv('cleaned-apr14-sep14.csv', index=True)

## Checking 2015 data

In [15]:
# import csv as a DataFrame
df2 = pd.read_csv('2015/uber-raw-data-jan15-june15.csv')

# check info
print(df2.dtypes)
print(df2.columns)
print(df2.shape)


Dispatching_base_num    object
Pickup_date             object
Affiliated_base_num     object
locationID               int64
dtype: object
Index(['Dispatching_base_num', 'Pickup_date', 'Affiliated_base_num',
       'locationID'],
      dtype='object')
(14270479, 4)


In [ ]:
# clean column names
df2.columns = df2.columns.str.lower()

# change 'pickup_date' to datetime
df2['pickup_date'] = pd.to_datetime(df2['pickup_date'])

# check changes
print(df2.columns)
print(df2.dtypes)

Index(['dispatching_base_num', 'pickup_date', 'affiliated_base_num',
       'locationid'],
      dtype='object')
dispatching_base_num            object
pickup_date             datetime64[ns]
affiliated_base_num             object
locationid                       int64
dtype: object


In [22]:
# make sure each month is full
monthly_stats = df2.groupby(df2['pickup_date'].dt.to_period('M')).agg({
    'pickup_date':['min', 'max', 'count']
})

print(monthly_stats)


                    pickup_date                             
                            min                 max    count
pickup_date                                                 
2015-01     2015-01-01 00:00:05 2015-01-31 23:59:59  1953801
2015-02     2015-02-01 00:00:00 2015-02-28 23:59:59  2263620
2015-03     2015-03-01 00:01:00 2015-03-31 23:59:00  2259773
2015-04     2015-04-01 00:01:00 2015-04-30 23:59:00  2280837
2015-05     2015-05-01 00:00:00 2015-05-31 23:59:00  2695553
2015-06     2015-06-01 00:02:00 2015-06-30 23:59:00  2816895


In [ ]:
# check nulls
print(df2.isnull().sum())

# lots of null in affiliated_base_num but that doesn't matter for our analysis

dispatching_base_num         0
pickup_date                  0
affiliated_base_num     162195
locationid                   0
dtype: int64


no changes made to 2015 stuff so doesn't need to be exported :)